In [ ]:
#| default_exp logger

In [ ]:
#| export
from __future__ import annotations

# Helpers

> ...

In [ ]:
import ipywidgets as W
W.Widget.close_all()  # we don't want stale widgets around when developing

# Prologue

In [ ]:
#| export
import json
import typing
from collections import deque
from functools import partialmethod
from typing import Callable

import fastcore.all as FC
import traitlets as T
from fasthtml.xtend import Style
from IPython.display import display
from olio.basic import bundle_path
from olio.basic import empty
from olio.common import shorten


In [ ]:
#| export
from bridget.bridge_widget import BridgeWidget
from bridget.bridge_widget import bundled
from bridget.helpers import displaydh
from bridget.helpers import HTML
from bridget.helpers import id_gen


In [ ]:
import os
import time
from pathlib import Path

from fastcore.test import *
from fastcore.xml import to_xml
from IPython.core.display import Javascript
from loguru import logger
from olio.common import setup_console
from olio.test import test_raises
from olio.widget import cleanupwidgets


In [ ]:
import bridget
from bridget.helpers import arun_command
from bridget.helpers import DEBUG


----

In [ ]:
#| exporti

new_id = id_gen()
BUNDLE_PATH = bundle_path(__name__)

In [ ]:
console, cprint = setup_console(140)

In [ ]:
#| exporti

_n = '\n'

----

# Canvas and Logger

# Canvas
> Places to display content

In [ ]:
#| export

class Canvas:
    def show(self, content=None, **kwargs):...
    def hide(self):...
    def add(self, content, **kwargs):...
    def clear(self, history:bool=True):...

## DHCanvas

In [ ]:
#| export

class DhCanvas(Canvas):
    _stl = 'border: 0.5px solid lightblue; overflow: auto;'
    def __init__(self, height:int=200):
        self.height, self._h, self._dh =  height, '', None
    @property
    def dh(self):
        if not self._dh: self._dh = displaydh()
        return self._dh
    def show(self, content=None, **kwargs):
        if self._dh: self.hide(); self._dh = None
        self.add(content or '', **kwargs)
    def hide(self): self.dh.update(HTML(''))
    def add(self, content, **kwargs):
        if content is not None:
            self._h = str(content) + self._h
            self.dh.update(HTML(f"<div style='max-height:{self.height}px;{self._stl}'>{self._h}</div>"), metadata=kwargs)
    def clear(self): self._h = ''; self.hide()

In [ ]:
cnv = DhCanvas(height=200)

In [ ]:
cnv.show("What's up, world!<br>")

In [ ]:
cnv.add("Take me to your leader.<br>")

In [ ]:
cnv.clear()

In [ ]:
cnv.add('<b>There and Back Again</b><br>')

In [ ]:
cnv.show()

In [ ]:
cnv.add('')

In [ ]:
cnv.add('\n')

In [ ]:
cnv.add('<br>')

## FCanvas (kernel)
> HTML element with a well-known id.

In [ ]:
#| export

FCanvas_css = '''
  @scope (.brd-logger) {
    :scope { 
      border: 1px solid steelblue; 
      overflow: auto; 
      font-family: monospace; 
      font-size: 13px; 
    }
    .ts { color:lightgray; }
  }
'''
FCanvas_stl = Style(FCanvas_css, id='brd-logger-stl')

In [ ]:
display(HTML(FCanvas_stl))

In [ ]:
class FCanvas(Canvas, T.HasTraits):
    height = T.Int(200).tag(sync=True)
    elid = T.Unicode('').tag(sync=True)
    def show(self, content=None, **kwargs):
        prev_elid = self.elid
        elid = new_id('brd-logger-')
        s = content or ''
        display(HTML(
            f"<div id='{elid}' class='brd-logger' "
            f"style='width: 100%; max-height: {self.height}px;'>{(s+'<br>') if not prev_elid and s else ''}</div>"), 
            metadata=kwargs)
        time.sleep(0.25)
        self.elid = elid
        if prev_elid: 
            display(Javascript(f"""
debugger;
const prevEl = document.getElementById('{prev_elid}');
let prevHtml = prevEl?.innerHTML ?? '';
if (prevEl) prevEl.style.display = 'none';
if (prevEl) prevEl.innerHTML = '';
{self._el()}; if (el) {{ el.innerHTML = prevHtml + '{s.replace("'", "\\'")}' + '<br>';
el.scrollTop = el.scrollHeight;}}
"""))
    def hide(self): display(Javascript(f"{self._el()} el.style.display = 'none'"))
    def add(self, content, **kwargs):
        if content is not None: display(Javascript(self._js(str(content).replace("'", "\\'"))))
    def clear(self): display(Javascript(f"{self._el()} el.innerHTML = ''"))

    def _el(self): return f"const el=document.getElementById('{self.elid}')"
    def _js(self, s):
        return f"{self._el()}; if (el) {{el.innerHTML += '{s}'; el.scrollTop = el.scrollHeight;}}"

In [ ]:
cnv = FCanvas(height=100)

In [ ]:
cnv.show()

In [ ]:
for c in 'abcdefgehijk': cnv.add(f"{c}<br>")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

...

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
cnv.add(f'<span class="ts">1234567890</span> <span class="msg">msg</span><br>')

<IPython.core.display.Javascript object>

In [ ]:
cnv.add("lorem ipsum dolor sit 'amet<br>")

<IPython.core.display.Javascript object>

In [ ]:
cnv.show()

<IPython.core.display.Javascript object>

In [ ]:
cnv.add({"a": 1})

<IPython.core.display.Javascript object>

This is obviously ugly and unwieldy. We need a more usable canvas, one that doen't rely in IPython display system.. The only solution in modern Jupyter envs is a widget.

## FCanvas (widget)

In [ ]:
os.environ['DEBUG_BRIDGET'] = 'True'

In [ ]:
#| exporti

fcanvas_js = BUNDLE_PATH / 'js/fcanvas.js'
fcanvas_esm = bundled(fcanvas_js)()

In [ ]:
fcanvas_esm = bundled(fcanvas_js)(debugger=DEBUG(), ts=True)

In [ ]:
#| export

class FCanvas(BridgeWidget, Canvas):
    # _esm = anysource(fcanvas_esm, '''
    _esm = '''
const { FCanvas } = await brdimport('./fcanvas.js');
export default { initialize({ model }) {
  let fcanvas = new FCanvas(model);
  model.set('_loaded', true); model.save_changes();
  return () => { fcanvas.model = null; fcanvas = null; console.log('Canvas cleanup!'); }
}}
'''
    _css = FCanvas_css

    height = T.Int(200).tag(sync=True)
    elid = T.Unicode('').tag(sync=True)
    _displayed = T.Bool(False).tag(sync=True)
    
    def __init__(self, height:int=200, elid:str='', **kwargs):
        if elid: display(self.tmpl(elid, height or self.height))
        super().__init__(height=height, elid=elid, **kwargs)

    @classmethod
    def new_elid(cls): return new_id('brd-logger')

    @classmethod
    def tmpl(cls, elid, height, content=''):
        return HTML(
            f"<div id='{elid}' class='brd-logger' "
            f"style='width: 100%; max-height: {height}px;'>{content}</div>")
    
    def displayed(self): return self._displayed
    def show(self, content=None, **kwargs):
        self.elid = elid = self.new_elid()
        display(self.tmpl(elid, self.height))
        res = self.send({'cmd': 'show', 'content': elid}, timeout=2)
        if res[0] != empty:  # type: ignore
            self._displayed = True
            if content is not None: self.add(content, **kwargs)
    def hide(self): self.send({'cmd': 'hide'})
    def add(self, content, **kwargs): self.send({ 'cmd': 'update', 'content':str(content), 'kw':kwargs})
    def clear(self): self.send({'cmd': 'clear'})
    
    def close(self, msg:str|None=None):
        if self.comm is not None:
            self.add('FCanvas Closed.<br>')
            self.send = self.show = FC.noop  # type: ignore
            super().close()

In [ ]:
cleanupwidgets('cnv')

cnv = FCanvas.create(height=100, timeout=3)
test_eq(cnv.loaded(), True)

moduleName='./fcanvas.js' buffers=[]


In [ ]:
cnv.show('hello')
# test_eq(cnv.displayed(), True)

In [ ]:
cnv.add(' bye<br>')

In [ ]:
cnv.clear()

In [ ]:
cnv.add('Goodbye to all that<br>')

In [ ]:
cnv.add({'a': str(Path('a'))})  # convert to json, no '<br>'

In [ ]:
cnv.add(f'<br><span class="ts">1234567890</span> <span class="msg">msg</span><br>')

In [ ]:
cnv.hide()
cnv.add('hideous!<br>')

In [ ]:
cnv.show()

In [ ]:
cnv.close()

In [ ]:
cleanupwidgets('cnv')

cnv = FCanvas.create(elid=FCanvas.new_elid())
test_eq(cnv.loaded(), True)

In [ ]:
cnv.add('Hi & Bi<br>')

In [ ]:
cnv.close()

# NBLogger

In [ ]:
#| export

_l2l = {'log':'INFO', 'error': 'ERROR', 'warn': 'WARN'}

class NBLogger:
    @property
    def canvas(self) -> Canvas:...
    def close_canvas(self):...
    def show(self, msg=None, clear:bool=False, **kwargs):...
    def msg(self, msg, clear:bool=False, **kwargs):...
    def clear_log(self):...
    def active(self, flag:bool|None=None, msg=None):...
    
    # hate stupid wiggly reds
    if typing.TYPE_CHECKING:
        def log(self, msg, clear:bool=False, **kwargs): ...
        def error(self, msg, clear:bool=False, **kwargs): ...
        def warn(self, msg, clear:bool=False, **kwargs): ...

    def __init_subclass__(cls) -> None:
        for n,l in _l2l.items(): setattr(cls, n, partialmethod(cls.msg, level=l))

class NoopLogger(NBLogger):
    def __getattr__(self, name:str): return FC.noop

## BasicLogger

In [ ]:
#| export

class BasicLogger(NBLogger):
    fmt, format, max_len = str, True, 140
    def __init__(self, msg=None, canvas:Canvas|None=None, height:int=200, show:bool=True, history:bool=True, **kwargs):
        self._canvas, self._active, self.height, self._closed = canvas, False, height, False
        self._msgs = deque(maxlen=30) if history else None
        if canvas : self.active(True)
        elif show: self.show()
        if msg: self.msg(msg, **kwargs)
    
    @property
    def canvas(self): return self._canvas
    def _setup_canvas(self, height:int): self._canvas = DhCanvas(height=height)
    def close_canvas(self):
        self.active(False)
        if self._canvas and hasattr(self._canvas, 'close'): self._canvas.close()  # type: ignore
        setattr(self, '_canvas', None); self._closed = True

    def show(self, msg=None, clear:bool=False, **kwargs):
        if self._closed: print('This logger is closed.'); return
        if self.active() and self.canvas:
            self.canvas.hide()
            if clear: self.clear_log()
        if self.canvas is None: self._setup_canvas(self.height)
        if self.canvas:
            self.canvas.show()
            self.active(True, msg)
    def _format(self, msg, fmt: Callable[[str], str]|str|None=None, truncate:bool=True, sep:str='<br>') -> str:
        fmt = fmt or self.fmt
        if self.max_len and truncate: s=shorten(msg, mode='r', limit=self.max_len)
        if isinstance(fmt, str): return fmt.format(s=s)
        return f"{fmt(s)}{sep}"
    def msg(self, msg, clear:bool=False, format:bool|None=True, 
            fmt: Callable[[str], str]|str|None=None, truncate:bool=True, sep:str='<br>', **kwargs):
        if not self.active(): return
        if clear: self.clear_log()
        if not msg: return
        format =  format if format is not None else self.format
        self.canvas.add(self._format(msg, fmt, truncate, sep) if format else msg, **kwargs)  # type: ignore
        if self._msgs is not None: self._msgs.append(msg)
    def clear_log(self):
        if self.canvas: self.canvas.clear()
        if self._msgs is not None: self._msgs.clear()
    def active(self, flag: bool|None=None, msg=None):
        if self.canvas is None: return False
        if flag is not None:
            if not flag and self._active: self.msg(msg)
            self._active = flag
            if flag: self.msg(msg)
        return self._active
    
    def history(self) -> tuple: return tuple(self._msgs or ())

In [ ]:
bl = BasicLogger('BasicLogger initialized', height=100)

In [ ]:
for i,x in enumerate(range(10)): bl.log(f'test{i}')

In [ ]:
bl.msg(f'''<span style="color: red;">{'red '*100}</span>''')

In [ ]:
bl.history()[-1]

'<span style="color: red;">red red red red red red red red red red red red red red red red red red red red red red red red red red red red red red red red red red red red red red red red red red red red red red red red red red red red red red red red red red red red red red red red red red red red red red red red red red red red red red red red red red red red red red red red red red red red red red red red red red red red </span>'

Wrong HTML: the message is shortened with the HTML format

In [ ]:
bl.log('green '*100, fmt='<span style="color: green;">{s}</span><br>')

In [ ]:
bl.show()

In [ ]:
bl.active(False, 'BasicLogger disabled')

False

In [ ]:
bl.error('test')

In [ ]:
bl.show("Enabled")

In [ ]:
bl.close_canvas()
bl.show()
bl.log('closed')
test_eq(bl.active(), False)

This logger is closed.


## FLogger

In [ ]:
#| export

class FLogger(BasicLogger):
    canvas: FCanvas
    def _setup_canvas(self, height:int): self._canvas = FCanvas.create(height=height, timeout=2)

In [ ]:
bl = FLogger('FLogger initialized', FCanvas.create(height=100, elid=FCanvas.new_elid(), timeout=2))

In [ ]:
for i,x in enumerate(range(5)): bl.log(f'test{i}')

In [ ]:
bl.error('red '*100, fmt=lambda s: f'<span style="color: red;">{s}</span>')

In [ ]:
bl.show()

In [ ]:
bl.active(False, 'FLogger disabled')

False

In [ ]:
bl.log('test')

In [ ]:
bl.close_canvas()
bl.log('closed')
bl.show()
test_eq(bl.active(), False)

This logger is closed.


## Loguru logger (WIP)

In [ ]:
#| exporti

level_colors = {
    "DEBUG": "#666666",    # gray
    "INFO": "#000000",     # black
    "SUCCESS": "#007700",  # green
    "WARNING": "#ff8800",  # orange
    "ERROR": "#ff0000",    # red
    "CRITICAL": "#880000", # dark red
}

In [ ]:
class LoguruBasicLogger(BasicLogger):
    def __init__(self): 
        super().__init__()
        self._fmt = FC.noop
    
    def write(self, message: str) -> None:
        if rec := getattr(message, 'record', None):
            level = rec['level'].name
        else: 
            for level in level_colors: 
                if level in message: break
        # message = f"<span style='color: {level_colors[level]}'>{message}</span>"
        # self.msg(message, fmt=lambda s:f"<span style='color: {level_colors[level]}'>{s}</span>")
        self.msg(message, fmt=f"<span style='color: {level_colors[level]}'>""{s}</span><br>")

In [ ]:
logger.remove()  # Remove default handler
handler_id = logger.add(
    (lbl := LoguruBasicLogger()).write, 
    format="{level} | {message}",  # Simple format, we'll add HTML in the sink
    colorize=False  # Disable ANSI colors
)

In [ ]:
logger.debug("This is a debug message")
logger.info("This is an info message")
logger.warning("This is a warning")
logger.error("This is an error")

In [ ]:
class LoguruBasicLogger(BasicLogger):
    def __init__(self):
        super().__init__()
        self.fmt = FC.noop
    
    def _format(self, msg, fmt: Callable[[str], str]|str|None=None, truncate:bool=True, sep:str='') -> str:
        rec = getattr(msg, 'record', json.loads(msg))
        return (
            f"<div style='display: flex; gap: 8px'>"
            f"<span style='color: #888'>{rec['time'].strftime('%H:%M:%S')}</span>"
            f"<span style='color: {level_colors[rec['level'].name]}'>{rec['level'].name:8}</span>"
            f"<span>{shorten(rec['message'], mode='r', limit=self.max_len or 140)}</span>"
            f"</div>"
        )

    def write(self, message:str) -> None:
        self.msg(message, sep='')
        # self.msg(formatted_msg)
    

def configure_logger(basic_logger: BasicLogger) -> int:
    """Configure loguru to use a specific BasicLogger instance."""
    logger.remove()
    return logger.add(
        basic_logger.write,  # type: ignore
        serialize=True  # This makes loguru pass a json to write()
    )

In [ ]:
lbl = LoguruBasicLogger()
handler_id = configure_logger(lbl)

In [ ]:
logger.debug("This is a debug message")
logger.info("This is an info message")
logger.warning("This is a warning")
logger.error("This is an error")

In [ ]:
#| exporti

@FC.patch
def setup_loguru_sink(self: BasicLogger, logger):
    def sink(message:str) -> None:
        rec = getattr(message, 'record', None) or json.loads(message)
        formatted_msg = (
            f"<div style='display: flex; gap: 8px'>"
            f"<span style='color: #888'>{rec['time'].strftime('%H:%M:%S.%f')}</span>"
            f"<span style='width: 4em; color: {level_colors[rec['level'].name]}'>{rec['level'].name.lower():8}</span>"
            # f"<span>{rec['level'].icon}</span>"
            f"<span>{shorten(rec['message'], mode='r', limit=self.max_len or 140)}</span>"
            f"</div>"
        )
        self.msg(formatted_msg, format=False)
    
    logger.remove()
    logger.add(
        sink, 
        # format="<span style='color: {level.color}'>{message}</span>",
        # colorize=True
        # serialize=True
    )
    return sink

In [ ]:
(my_logger := BasicLogger()).setup_loguru_sink(logger);  # type: ignore

In [ ]:
logger.debug("This is a debug message")
logger.info("This is an info message")
logger.warning("This is a warning")
logger.error("This is an error")

In [ ]:
(my_flogger := FLogger()).setup_loguru_sink(logger);  # type: ignore

In [ ]:
logger.debug("This is a debug message")
logger.info("This is an info message")
logger.warning("This is a warning")
logger.error("This is an error")

In [ ]:
my_flogger.close_canvas()

# Colophon
----

In [ ]:
import fastcore.all as FC
import nbdev
from nbdev.clean import nbdev_clean
from olio.basic import bundle_path
from bridget.bridge_widget import bundled

In [ ]:
if FC.IN_NOTEBOOK:
    BUNDLE_PATH = bundle_path(__name__)
    for f in ['fcanvas']: bundled(BUNDLE_PATH / f'js/{f}.js')()
    nb_path = '11_logger.ipynb'
    # nbdev_clean(nb_path)
    nbdev.nbdev_export(nb_path)